In [50]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from sklearn.linear_model import LinearRegression
import time
import datetime

In [51]:
tripdata = pd.read_csv("../../dataset/final_bike_sharing.csv")

In [52]:
tripdata.head(3)

,index,tripduration,start_station_id,start_station_name,start_lat,start_lon,end_station_id,end_station_name,end_lat,end_lon,...,conditions,date,dist,birthyear,years_old,holiday,day,month,seasons,gender
0,259617,1.02,others,Manila & 1st,40.721651,-74.042884,others,Manila & 1st,40.721651,-74.042884,...,Clear,2018-09-05,1.783012e-12,1994,24,working_day,weekday,September,autumn,male
1,283363,1.02,others,Heights Elevator,40.748716,-74.040443,others,Heights Elevator,40.748716,-74.040443,...,cloudy_rain,2018-10-06,1.782506e-12,1994,24,working_day,weekend,October,autumn,male
2,169168,1.02,others,Van Vorst Park,40.718489,-74.047727,es_3185,City Hall,40.717732,-74.043845,...,Clear,2018-07-09,3.384331e-01,1963,55,working_day,weekday,July,summer,female


In [53]:
tripdata.columns

Index(['index', 'tripduration', 'start_station_id', 'start_station_name',
       'start_lat', 'start_lon', 'end_station_id', 'end_station_name',
       'end_lat', 'end_lon', 'bikeid', 'usertype', 'hour', 'min', 'temp',
       'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'snow',
       'snowdepth', 'windspeed', 'winddir', 'sealevelpressure', 'cloudcover',
       'visibility', 'solarradiation', 'uvindex', 'conditions', 'date', 'dist',
       'birthyear', 'years_old', 'holiday', 'day', 'month', 'seasons',
       'gender'],
      dtype='object')

## Selecting variables according to P values

In [54]:
selected_features = ['tripduration','start_lat','start_lon','end_lat','end_lon','usertype','hour',
                     'temp','feelslike','dew','snowdepth','winddir','sealevelpressure','visibility','solarradiation',
                     'conditions','dist','birthyear','holiday','day','month','gender']

In [55]:
df = tripdata[selected_features]

In [56]:
def split_dataset(df):
    X = df.drop('tripduration', axis=1)
    y = df['tripduration']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print('Train', X_train.shape, y_train.shape)
    print('Test', X_test.shape, y_test.shape)
    return X_train, X_test, y_train, y_test

### AMputation with 10% NA values

In [57]:
## get the sample size such as 5%, 10%,20% etc
def get_sample_size(df, perc=1):
    return round(perc/100 * df.shape[0])

def get_an_index(index):
    li = []
    for i in index:
        li.append(i)
    return li   

def ampute_nan_values(X_train,y_train,col_name, perc):
    random.seed(100)
    index = get_an_index(X_train.index)
    sample_size = get_sample_size(X_train, perc)
    print("sample size is =", sample_size)
    selected_index = random.sample(index, sample_size)
    for i in selected_index:
        X_train.loc[i, col_name]= np.NaN
    return X_train, y_train

###====================####
def convert_to_dataframe(y_train):
    return pd.DataFrame(y_train)

def convert_to_series(y_train):
    return y_train.squeeze()

def ampute_nan_values_drop(X_train,y_train,col_name, perc):
    random.seed(100)
    index = get_an_index(X_train.index)
    sample_size = get_sample_size(X_train, perc)
    print("sample size is =", sample_size)
    selected_index = random.sample(index, sample_size)
    y_train = convert_to_dataframe(y_train)
    for i in selected_index:
        y_train.loc[i] = np.NaN
        X_train.loc[i,col_name]= np.NaN
    y_train = convert_to_series(y_train)
    return X_train, y_train

In [58]:
def sample_imputation_X(var,X_train,X_test):
    # extract a random sample
    random_sample_train = X_train[var].dropna().sample(X_train[var].isnull().sum(), random_state=0)
    random_sample_test = X_test[var].dropna().sample(X_test[var].isnull().sum(), random_state=0)
     # re-index the randomly extracted sample
    random_sample_train.index = X_train[X_train[var].isnull()].index
    random_sample_test.index = X_test[X_test[var].isnull()].index

    # replace the NA
    X_train.loc[X_train[var].isnull(), var] = random_sample_train
    X_test.loc[X_test[var].isnull(), var] = random_sample_test
    return X_train, X_test

In [59]:
def ampute_each_variables(variable, method,X_train,y_train,X_test,y_test):
    if method == "remove":
        X_train , y_train = ampute_nan_values_drop(X_train,y_train,variable,perc=20)
        print('Train set after Amputation', X_train.shape, y_train.shape)
        print("y_train Amputed value count of varaible ", y_train.isnull().sum())
        print("X_train Amputed value count of varaible ", X_train.isnull().sum()[variable])
        X_test , y_test = ampute_nan_values_drop(X_test,y_test,variable,perc=20)
        print('Test set after Amputation', X_test.shape, y_test.shape)
        print("y_test Amputed value count of varaible ", y_test.isnull().sum())
        print("X_test Amputed value count of varaible ", X_test.isnull().sum()[variable])
    else:
        X_train , y_train = ampute_nan_values(X_train,y_train,variable,perc=20)
        print('Train set after Amputation', X_train.shape, y_train.shape)
        print("X_train Amputed value count of varaible ", X_train.isnull().sum()[variable])
        X_test , y_test = ampute_nan_values(X_test,y_test,variable,perc=20)
        print('Test set after Amputation', X_test.shape, y_test.shape)
        print("X_test Amputed value count of varaible ", X_test.isnull().sum()[variable])
    start = datetime.datetime.now()
    if method == "remove":
        print("Simulate Imputation for NA")
        X_train.dropna(axis=0, inplace=True)
        y_train.dropna(inplace=True)
        print('Train after NA value removal Imputation', X_train.shape, y_train.shape)
        X_test.dropna(axis=0, inplace=True)
        y_test.dropna(inplace=True)
        print('Test after NA value removal Imputation', X_test.shape, y_test.shape)
    if method == "mode":
        print("Simulate Imputation for Mode")
        X_train.fillna(X_train[variable].mode()[0], inplace=True)
        print('Train after Mode Imputation', X_train.shape, y_train.shape)
        X_test.fillna(X_test[variable].mode()[0], inplace=True)
        print('Test after Mode Imputation', X_test.shape, y_test.shape)
    if method == "mean":
        print("Simulate Imputation for Mean")
        X_train.fillna(X_train[variable].mean(), inplace=True)
        print('Train after Mean Imputation', X_train.shape, y_train.shape)
        X_test.fillna(X_test[variable].mean(), inplace=True)
        print('Test after Mean Imputation', X_test.shape, y_test.shape)
    if method == "sample":
        print("Simple Imputation using sampling")
        X_train, X_test = sample_imputation_X(variable, X_train, X_test)
    end = datetime.datetime.now()
    comp_time = (end-start)
    print(f"Time taken simulate for variable {var} = ", comp_time)
    list_comp_time.append(comp_time)      
    return X_train , y_train, X_test , y_test

In [60]:
def one_hot_encoding(X_train, X_test):
    categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
    numerical_cols = X_train.select_dtypes(include=['int', 'float']).columns.tolist()
    ohe = OneHotEncoder(drop='first', sparse=False)
    X_train_encoded = ohe.fit_transform(X_train[categorical_features])
    X_test_encoded = ohe.fit_transform(X_test[categorical_features])
    X_train_new = np.hstack((X_train[numerical_cols].values, X_train_encoded))
    X_test_new = np.hstack((X_test[numerical_cols].values, X_test_encoded))
    return X_train_new, X_test_new

In [61]:
def model_evaluation(X_train,y_train):
    print(datetime.datetime.now())
    model_lr = LinearRegression(fit_intercept=True).fit(X_train, y_train) 
    print(datetime.datetime.now())
    return model_lr

In [62]:
features_for_amputation = ['start_lat','start_lon','end_lat','end_lon','usertype','hour',
                     'temp','feelslike','dew','snowdepth','winddir','sealevelpressure','visibility','solarradiation',
                     'conditions','dist','birthyear','holiday','day','month','gender']

## 1. Drop NA values 

In [63]:
model_metrics ={}
list_comp_time = []
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"remove",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    model_metrics[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred)}
    del X_train
    del X_test
    del y_train
    del y_test

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 50469
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  50469
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (63087,)
y_test Amputed value count of varaible  12617
X_test Amputed value count of varaible  12617
Simulate Imputation for NA
Train after NA value removal Imputation (201877, 21) (201877,)
Test after NA value removal Imputation (50470, 21) (50470,)
Time taken simulate for variable start_lat =  0:00:00.325220
2023-03-25 13:24:18.439432
2023-03-25 13:24:18.769836
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 50469
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  50469
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 

sample size is = 50469
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  50469
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (63087,)
y_test Amputed value count of varaible  12617
X_test Amputed value count of varaible  12617
Simulate Imputation for NA
Train after NA value removal Imputation (201877, 21) (201877,)
Test after NA value removal Imputation (50470, 21) (50470,)
Time taken simulate for variable visibility =  0:00:00.335924
2023-03-25 13:26:15.731342
2023-03-25 13:26:16.047628
Now running model is .... solarradiation
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 50469
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  50469
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (63087,)
y_test Amputed value count of varaible  12617
X_test Amputed value count

In [64]:
print("averge time to run drop is : ", 0.3261990, "seconds")
list_comp_time

averge time to run drop is :  0.35481595 seconds


[datetime.timedelta(microseconds=325220),
 datetime.timedelta(microseconds=327570),
 datetime.timedelta(microseconds=287666),
 datetime.timedelta(microseconds=320198),
 datetime.timedelta(microseconds=309753),
 datetime.timedelta(microseconds=356252),
 datetime.timedelta(microseconds=315860),
 datetime.timedelta(microseconds=283958),
 datetime.timedelta(microseconds=291720),
 datetime.timedelta(microseconds=319161),
 datetime.timedelta(microseconds=332984),
 datetime.timedelta(microseconds=287330),
 datetime.timedelta(microseconds=335924),
 datetime.timedelta(microseconds=324310),
 datetime.timedelta(microseconds=309665),
 datetime.timedelta(microseconds=382625),
 datetime.timedelta(microseconds=388179),
 datetime.timedelta(microseconds=379390),
 datetime.timedelta(microseconds=339898),
 datetime.timedelta(microseconds=309571),
 datetime.timedelta(microseconds=322946)]

In [65]:
model_metrics

{'start_lat': {'MAE': 1.5707629260207865,
  'MSE': 4.985157143314904,
  'RMSE': 2.2327465470390733,
  'R2': 0.5282643786583994},
 'start_lon': {'MAE': 1.5707629260207865,
  'MSE': 4.985157143314904,
  'RMSE': 2.2327465470390733,
  'R2': 0.5282643786583994},
 'end_lat': {'MAE': 1.5707629260207865,
  'MSE': 4.985157143314904,
  'RMSE': 2.2327465470390733,
  'R2': 0.5282643786583994},
 'end_lon': {'MAE': 1.5707629260207865,
  'MSE': 4.985157143314904,
  'RMSE': 2.2327465470390733,
  'R2': 0.5282643786583994},
 'usertype': {'MAE': 1.5707629260207865,
  'MSE': 4.985157143314904,
  'RMSE': 2.2327465470390733,
  'R2': 0.5282643786583994},
 'hour': {'MAE': 1.5707629260207865,
  'MSE': 4.985157143314904,
  'RMSE': 2.2327465470390733,
  'R2': 0.5282643786583994},
 'temp': {'MAE': 1.5707629260207865,
  'MSE': 4.985157143314904,
  'RMSE': 2.2327465470390733,
  'R2': 0.5282643786583994},
 'feelslike': {'MAE': 1.5707629260207865,
  'MSE': 4.985157143314904,
  'RMSE': 2.2327465470390733,
  'R2': 0.52

### Mode Replacement

In [66]:
model_metrics_mode ={}
list_comp_time = []
for var in features_for_amputation:
    print("Now model is running for varaible ====", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mode",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    model_metrics_mode[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred)}
    del X_train
    del X_test
    del y_train
    del y_test

Now model is running for varaible ==== start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 50469
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  12617
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (63087, 21) (63087,)
Time taken simulate for variable start_lat =  0:00:00.373962
2023-03-25 13:27:48.855218
2023-03-25 13:27:49.204599
Now model is running for varaible ==== start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 50469
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  12617
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346

Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  12617
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (63087, 21) (63087,)
Time taken simulate for variable conditions =  0:00:00.374766
2023-03-25 13:29:27.639154
2023-03-25 13:29:27.988419
Now model is running for varaible ==== dist
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 50469
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  12617
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (63087, 21) (63087,)
Time taken simulate for variable dist =  0:00:00.326457
2023-03-25 13:29:34.2094

In [67]:
print("averge time to run drop is : ", 0.3159189, "seconds")
list_comp_time

averge time to run drop is :  0.35481595 seconds


[datetime.timedelta(microseconds=373962),
 datetime.timedelta(microseconds=252103),
 datetime.timedelta(microseconds=304059),
 datetime.timedelta(microseconds=255564),
 datetime.timedelta(microseconds=277120),
 datetime.timedelta(microseconds=378989),
 datetime.timedelta(microseconds=293308),
 datetime.timedelta(microseconds=294997),
 datetime.timedelta(microseconds=308774),
 datetime.timedelta(microseconds=306489),
 datetime.timedelta(microseconds=375584),
 datetime.timedelta(microseconds=328004),
 datetime.timedelta(microseconds=316134),
 datetime.timedelta(microseconds=272472),
 datetime.timedelta(microseconds=374766),
 datetime.timedelta(microseconds=326457),
 datetime.timedelta(microseconds=291683),
 datetime.timedelta(microseconds=396167),
 datetime.timedelta(microseconds=281883),
 datetime.timedelta(microseconds=325628),
 datetime.timedelta(microseconds=300154)]

In [68]:
model_metrics_mode

{'start_lat': {'MAE': 1.5722130252690756,
  'MSE': 5.000209201674977,
  'RMSE': 2.236114755927114,
  'R2': 0.5271793557173245},
 'start_lon': {'MAE': 1.5731200523922648,
  'MSE': 5.004431807531456,
  'RMSE': 2.2370587402952693,
  'R2': 0.5267800653794436},
 'end_lat': {'MAE': 1.57225520655082,
  'MSE': 5.000424067378656,
  'RMSE': 2.236162799837851,
  'R2': 0.5271590379793359},
 'end_lon': {'MAE': 1.5733007420152174,
  'MSE': 5.001869650722917,
  'RMSE': 2.2364860050362303,
  'R2': 0.5270223433690446},
 'usertype': {'MAE': 1.5780241854409498,
  'MSE': 5.032746222635772,
  'RMSE': 2.243378305733514,
  'R2': 0.5241026494050028},
 'hour': {'MAE': 1.5737265209413012,
  'MSE': 5.007616314859015,
  'RMSE': 2.2377703892175833,
  'R2': 0.5264789378174537},
 'temp': {'MAE': 1.5726022149666066,
  'MSE': 5.000988054240213,
  'RMSE': 2.2362889022307053,
  'R2': 0.5271057072844605},
 'feelslike': {'MAE': 1.5724426255795068,
  'MSE': 5.00025727555317,
  'RMSE': 2.2361255053223577,
  'R2': 0.52717480

### Mean Replacement

In [69]:
variables_for_mean_amputation = ['start_lat','start_lon','end_lat','end_lon','hour',
                                 'temp','feelslike','dew','snowdepth','winddir','visibility','sealevelpressure',
                                 'solarradiation','dist','birthyear']

In [70]:
model_metrics_mean ={}
list_comp_time = []
for var in variables_for_mean_amputation:
    print("Now model is running for varaible ====", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mean",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    model_metrics_mean[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred)}
    del X_train
    del X_test
    del y_train
    del y_test

Now model is running for varaible ==== start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 50469
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  12617
Simulate Imputation for Mean
Train after Mean Imputation (252346, 21) (252346,)
Test after Mean Imputation (63087, 21) (63087,)
Time taken simulate for variable start_lat =  0:00:00.224072
2023-03-25 13:30:15.008764
2023-03-25 13:30:15.338650
Now model is running for varaible ==== start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 50469
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  12617
Simulate Imputation for Mean
Train after Mean Imputation (252346, 21) (252346

Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  12617
Simulate Imputation for Mean
Train after Mean Imputation (252346, 21) (252346,)
Test after Mean Imputation (63087, 21) (63087,)
Time taken simulate for variable birthyear =  0:00:00.380213
2023-03-25 13:32:30.706592
2023-03-25 13:32:31.047175


In [71]:
print("averge time to run mean is : ", 0.387038, "seconds")
list_comp_time

averge time to run drop is :  0.35481595 seconds


[datetime.timedelta(microseconds=224072),
 datetime.timedelta(microseconds=412862),
 datetime.timedelta(microseconds=493635),
 datetime.timedelta(microseconds=393367),
 datetime.timedelta(microseconds=435862),
 datetime.timedelta(microseconds=433961),
 datetime.timedelta(microseconds=632215),
 datetime.timedelta(microseconds=353454),
 datetime.timedelta(microseconds=483626),
 datetime.timedelta(microseconds=330152),
 datetime.timedelta(microseconds=386888),
 datetime.timedelta(microseconds=256964),
 datetime.timedelta(microseconds=325187),
 datetime.timedelta(microseconds=263115),
 datetime.timedelta(microseconds=380213)]

In [72]:
model_metrics_mean

{'start_lat': {'MAE': 1.5722550153708958,
  'MSE': 5.0002226217131,
  'RMSE': 2.2361177566740755,
  'R2': 0.5271780867162057},
 'start_lon': {'MAE': 1.573040903172298,
  'MSE': 5.004207862632298,
  'RMSE': 2.23700868631132,
  'R2': 0.5268012416477221},
 'end_lat': {'MAE': 1.57228429465235,
  'MSE': 5.000370928083709,
  'RMSE': 2.2361509180025636,
  'R2': 0.5271640628402288},
 'end_lon': {'MAE': 1.573303497299732,
  'MSE': 5.001814060824874,
  'RMSE': 2.236473577045987,
  'R2': 0.5270275999593889},
 'hour': {'MAE': 1.5732184751717613,
  'MSE': 5.003379389861619,
  'RMSE': 2.23682350440566,
  'R2': 0.5268795821757688},
 'temp': {'MAE': 1.5725953614455166,
  'MSE': 5.000956942319607,
  'RMSE': 2.236281946070219,
  'R2': 0.5271086492330375},
 'feelslike': {'MAE': 1.5724520390696803,
  'MSE': 5.000300389737768,
  'RMSE': 2.2361351456783125,
  'R2': 0.5271707329583837},
 'dew': {'MAE': 1.5722639380866297,
  'MSE': 4.9995794149944075,
  'RMSE': 2.2359739298557146,
  'R2': 0.5272389084544316},

### Sampling Replacement

In [73]:
model_metrics_sample ={}
list_comp_time = []
for var in features_for_amputation:
    print("Now model is running for varaible ====", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"sample",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    model_metrics_sample[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred)}
    del X_train
    del X_test
    del y_train
    del y_test

Now model is running for varaible ==== start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 50469
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  12617
Simple Imputation using sampling
Time taken simulate for variable start_lat =  0:00:00.039387
2023-03-25 13:32:37.586635
2023-03-25 13:32:37.965528
Now model is running for varaible ==== start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 50469
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  12617
Simple Imputation using sampling
Time taken simulate for variable start_lon =  0:00:00.042631
2023-03-25 13:32:44.135168
2023-03-25 13:32:44.533066
Now model is running for

2023-03-25 13:34:22.225789
2023-03-25 13:34:22.578439
Now model is running for varaible ==== holiday
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 50469
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  12617
Simple Imputation using sampling
Time taken simulate for variable holiday =  0:00:00.088169
2023-03-25 13:34:28.685805
2023-03-25 13:34:29.041442
Now model is running for varaible ==== day
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 50469
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  50469
sample size is = 12617
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  12617
Simple Imputation using sampling
Time taken simulate for variable day =  0:00:00.073600
2023-03-25 13:34:34.961156
2023-03-25 13

In [74]:
print("averge time to run sample is : ", 0.523778, "seconds")
list_comp_time

averge time to run drop is :  0.35481595 seconds


[datetime.timedelta(microseconds=39387),
 datetime.timedelta(microseconds=42631),
 datetime.timedelta(microseconds=40990),
 datetime.timedelta(microseconds=26997),
 datetime.timedelta(microseconds=109602),
 datetime.timedelta(microseconds=51997),
 datetime.timedelta(microseconds=28124),
 datetime.timedelta(microseconds=44106),
 datetime.timedelta(microseconds=44651),
 datetime.timedelta(microseconds=32076),
 datetime.timedelta(microseconds=29009),
 datetime.timedelta(microseconds=34017),
 datetime.timedelta(microseconds=28543),
 datetime.timedelta(microseconds=28023),
 datetime.timedelta(microseconds=89554),
 datetime.timedelta(microseconds=29051),
 datetime.timedelta(microseconds=72071),
 datetime.timedelta(microseconds=88169),
 datetime.timedelta(microseconds=73600),
 datetime.timedelta(microseconds=80757),
 datetime.timedelta(microseconds=86579)]

In [75]:
model_metrics_sample

{'start_lat': {'MAE': 1.5721858537040991,
  'MSE': 5.000123563157097,
  'RMSE': 2.2360956068909705,
  'R2': 0.5271874537103416},
 'start_lon': {'MAE': 1.5735238600154633,
  'MSE': 5.006689839171069,
  'RMSE': 2.2375633709843994,
  'R2': 0.5265665455182555},
 'end_lat': {'MAE': 1.5722405149892742,
  'MSE': 5.001055386088714,
  'RMSE': 2.2363039565516836,
  'R2': 0.5270993403732563},
 'end_lon': {'MAE': 1.5737372496144444,
  'MSE': 5.003683882167789,
  'RMSE': 2.23689156692223,
  'R2': 0.5268507893307945},
 'usertype': {'MAE': 1.5826559045403263,
  'MSE': 5.055900137053465,
  'RMSE': 2.2485328854729842,
  'R2': 0.5219132112653002},
 'hour': {'MAE': 1.5744080137214291,
  'MSE': 5.009882932063067,
  'RMSE': 2.2382767773586596,
  'R2': 0.5262646061038121},
 'temp': {'MAE': 1.572625778070869,
  'MSE': 5.001133459931957,
  'RMSE': 2.2363214124834463,
  'R2': 0.5270919576971773},
 'feelslike': {'MAE': 1.5724446142813107,
  'MSE': 5.000302598539022,
  'RMSE': 2.2361356395663976,
  'R2': 0.52717